# Day 17: Trick Shot

https://adventofcode.com/2021/day/17

## Part 1

You finally decode the Elves' message. HI, the message says. You continue searching for the sleigh keys.

Ahead of you is what appears to be a large ocean trench. Could the keys have fallen into it? You'd better send a probe to investigate.

The probe launcher on your submarine can fire the probe with any integer velocity in the x (forward) and y (upward, or downward if negative) directions. For example, an initial x,y velocity like 0,10 would fire the probe straight up, while an initial velocity like 10,-1 would fire the probe forward at a slight downward angle.

The probe's x,y position starts at 0,0. Then, it will follow some trajectory by moving in steps. On each step, these changes occur in the following order:
```
    The probe's x position increases by its x velocity.
    The probe's y position increases by its y velocity.
    Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
    Due to gravity, the probe's y velocity decreases by 1.
```
For the probe to successfully make it into the trench, the probe must be on some trajectory that causes it to be within a target area after any step. The submarine computer has already calculated this target area (your puzzle input). For example:

target area: x=20..30, y=-10..-5

This target area means that you need to find initial x,y velocity values such that after any step, the probe's x position is at least 20 and at most 30, and the probe's y position is at least -10 and at most -5.

Given this target area, one initial velocity that causes the probe to be within the target area after any step is 7,2:
```
.............#....#............
.......#..............#........
...............................
S........................#.....
...............................
...............................
...........................#...
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTT#TT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
```
In this diagram, S is the probe's initial position, 0,0. The x coordinate increases to the right, and the y coordinate increases upward. In the bottom right, positions that are within the target area are shown as T. After each step (until the target area is reached), the position of the probe is marked with #. (The bottom-right # is both a position the probe reaches and a position in the target area.)

Another initial velocity that causes the probe to be within the target area after any step is 6,3:
```
...............#..#............
...........#........#..........
...............................
......#..............#.........
...............................
...............................
S....................#.........
...............................
...............................
...............................
.....................#.........
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................T#TTTTTTTTT
....................TTTTTTTTTTT
```
Another one is 9,0:
```
S........#.....................
.................#.............
...............................
........................#......
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTT#
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
```
One initial velocity that doesn't cause the probe to be within the target area after any step is 17,-4:
```
S..............................................................
...............................................................
...............................................................
...............................................................
.................#.............................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT..#.............................
....................TTTTTTTTTTT................................
...............................................................
...............................................................
...............................................................
...............................................................
................................................#..............
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
..............................................................#
```
The probe appears to pass through the target area, but is never within it after any step. Instead, it continues down and to the right - only the first few steps are shown.

If you're going to fire a highly scientific probe out of a super cool probe launcher, you might as well do it with style. How high can you make the probe go while still reaching the target area?

In the above example, using an initial velocity of 6,9 is the best you can do, causing the probe to reach a maximum y position of 45. (Any higher initial y velocity causes the probe to overshoot the target area entirely.)

Find the initial velocity that causes the probe to reach the highest y position and still eventually be within the target area after any step. What is the highest y position it reaches on this trajectory?

In [1]:
from IPython.display import Markdown

In [2]:
#infile = 'test_input.txt'
infile = 'input.txt'

with open(infile, 'r') as fid:
    line = fid.readline().strip()

xrange, yrange = line[line.find('x='):].split(',')

In [3]:
xrange = [int(x) for x in xrange.lstrip('x=').split('..')]
yrange = [int(y) for y in yrange.strip().lstrip('y=').split('..')]
xrange, yrange

([248, 285], [-85, -56])

In [4]:
def calc_positions(vx, vy, xrange=xrange, yrange=yrange):
    """
    Return list of trajectory positions starting from (0,0)
    until the x position either exceeds max(xrange) or the
    y position drops below min(yrange)
    
    If trajectory will necessarily come up short, return None
    """
    pos = (0, 0)
    poslist = [pos]
    while True:
        # The probe's x position increases by its x velocity.
        xpos = pos[0] + vx
        # The probe's y position increases by its y velocity.
        ypos = pos[1] + vy
        # Due to drag, the probe's x velocity changes by 1 toward
        # the value 0; that is, it decreases by 1 if it is greater than 0,
        # increases by 1 if it is less than 0, or does not change if it is already 0.
        if 0 != vx:
            vx = (abs(vx) - 1) * (abs(vx) // vx)
        # Due to gravity, the probe's y velocity decreases by 1.
        vy -= 1
        pos = (xpos, ypos)
        poslist.append(pos)
        # Check for out of bounds
        if xpos > max(xrange):
            return poslist
        if ypos < min(yrange):
            return poslist
        # Check that there's any chance probe can still hit target...
        # If it has zero velocity and xpos is outside of xrange
        # then it can never get there
        if xpos < min(xrange) and xpos > max(xrange) and 0 == vx:
            return None
        # Keep on calculating trajectory
        #print("({},{});".format(xpos, ypos), end='')


In [62]:
def check_coordhit(xpos, ypos, xrange=xrange, yrange=yrange):
    """
    Return True of the supplied xpos, ypos hit target
    window, otherwise False.
    """
    if min(xrange) <= xpos and max(xrange) >= xpos:
        xwindow = True
    else:
        xwindow = False
    if min(yrange) <= ypos and max(yrange) >= ypos:
        ywindow = True
    else:
        ywindow = False
    if xwindow and ywindow:
        return True
    else:
        return False

def calc_hit(vx, vy, xrange=xrange, yrange=yrange):
    """
    Return a dictionary for the probe results:
    {'hit': (xpos, ypos), 'ymax': ymax}
    If probe misses, value for 'hit' key will be None
    but if it hits, will return the first (xpos, ypos)
    in the target window
    Value for 'ymax' will be max ypos achieved
    """
    xpos = 0
    ypos = 0
    ymax = ypos
    while True:
        # The probe's x position increases by its x velocity.
        xpos += vx
        # The probe's y position increases by its y velocity.
        ypos += vy
        if ypos > ymax:
            ymax = ypos
        # Due to drag, the probe's x velocity changes by 1 toward
        # the value 0; that is, it decreases by 1 if it is greater than 0,
        # increases by 1 if it is less than 0, or does not change if it is already 0.
        if 0 != vx:
            vx = (abs(vx) - 1) * (abs(vx) // vx)
        # Due to gravity, the probe's y velocity decreases by 1.
        vy -= 1
        # Check for out of bounds
        if xpos > max(xrange):
            return {'hit': False, 'xpos': xpos, 'ypos': ypos, 'ymax': ymax}
        if ypos < min(yrange):
            return {'hit': False, 'xpos': xpos, 'ypos': ypos, 'ymax': ymax}
        # Check that there's any chance probe can still hit target...
        # If it has zero velocity and xpos is outside of xrange
        # then it can never get there
        ######if xpos < min(xrange) and xpos > max(xrange) and 0 == vx:
        ######    # No chance, probe has stalled outside of x-window
        ######    return {'hit': False, 'xpos': xpos, 'ypos': ypos, 'ymax': ymax}
        # Still here? Check if probe hit target
        if check_coordhit(xpos, ypos):
            # Hit!
            return {'hit': True, 'xpos': xpos, 'ypos': ypos, 'ymax': ymax}
        else:
            # No hit this time, keep trying
            pass

In [63]:
# Should hit at (28, -7):
#calc_positions(7, 2)
#calc_hit(7, 2)

# Should hit at (21, -9):
#calc_positions(6, 3)
#calc_hit(6, 3)

# Should hit at (30, -6):
#calc_positions(9, 0)
#calc_hit(9, 0)

# Should miss
#calc_positions(17, -4)
#calc_hit(17, -4)

# Should hit... achieve max y position
# of 45 for test input
#calc_positions(6, 9)
#calc_hit(6, 9)

In [64]:
def check_hit(poslist, xrange=xrange, yrange=yrange):
    """
    Check whether any of the positions in poslist hit
    the target in xrange
    """
    for x, y in poslist:
        if min(xrange) <= x and max(xrange) >= x:
            xwindow = True
        else:
            xwindow = False
        if min(yrange) <= y and max(yrange) >= y:
            ywindow = True
        else:
            ywindow = False
        if xwindow and ywindow:
            return x, y
    return False

In [65]:
#check_hit(calc_positions(7,2))
#check_hit(calc_positions(6,3))
#check_hit(calc_positions(9,0))
#check_hit(calc_positions(17,-4))

In [66]:
def get_maxy(poslist):
    """
    From supplied position list, return the max
    trajectory height
    """
    maxy = poslist[0][1]
    y0 = maxy
    for pos in poslist:
        if pos[1] < y0:
            # Object is dropping and we're done
            break
        if pos[1] > maxy:
            maxy = pos[1]
            y0 = maxy
    return maxy

In [67]:
#get_maxy(calc_positions(7, 2))
get_maxy(calc_positions(6, 3))
#get_maxy(calc_positions(9, 0))
#get_maxy(calc_positions(17, -4))
#get_maxy(calc_positions(6, 9))

6

In [68]:
# Loop while incrementing y value first, xvalue second.
# As soon as x value stops
# increasing and is less than min(xrange) stop, this cannot be
# a solution

# For a given vy, increment xv until target is hit;
# continue until target is no longer hit and return
# the maxy for all the hits
# If target is not hit before xv goes to zero, terminate

#vy = 9
#vy = -20
#vx = 1

def get_ymax_at_vy(vy, xrange=xrange, yrange=yrange):
    """
    For a supplied vy, try all plausible vx's and return
    the greatest possible ymax
    """
    vx = 1
    inbounds = False
    leftbounds = False
    ymax = max(yrange)
    overshot = None
    overshotdelta0 = None
    overshotdelta = 0
    undershot = None
    maxovershoots = 10000
    overshootcount = 0
    maxundershoots = 10000
    undershootcount = 0
    # Need to handle case where there's NEVER a hit
    while True:
        # Compute hit dictionary
        hdict = calc_hit(vx, vy)
        if not hdict['hit']:
            if inbounds:
                # Previous launches were in bounds but are
                # now out of bounds. Theres no hope of
                # ever hitting the target by changing vx
                ##########break
                pass
            # Increase vx and try again
            vx += 1
        else:
            # Target was hit
            ymax = hdict['ymax'] if hdict['ymax'] > ymax else ymax
            vx += 1
            # Probe targetting is now in bounds
            inbounds = True
            continue
        # In case we never hit, consider just the xrange...
        # Once the xrange boundaries are traversed then there's
        # never a chance again to hit the target
        if hdict['xpos'] > xrange[1]:
            # Target is overshot
            '''
            if overshot is not None and hdict['xpos'] > overshot:
                # Target has been overshot even further.
                overshotdelta = hdict['xpos'] - overshot
                if overshotdelta0 is None:
                    overshotdelta0 = overshotdelta
                elif overshotdelta > overshotdelta0:
                    # Will never hit
                    break
                overshot = hdict['xpos']
            else:
                overshot = hdict['xpos']
            '''
            overshootcount += 1
            if overshootcount >= maxovershoots:
                break
        if hdict['xpos'] < xrange[0]:
            # Target is undershot
        #    if undershot is not None and hdict['xpos'] < undershot:
        #        # Target has been undershot in wrong dirextion.
        #        # Will never hit
        #        break
        #    else:
        #        undershot = hdict['xpos']
            undershootcount += 1
            if undershootcount >= maxundershoots:
                break
    return ymax

#get_ymax_at_vy(9)

def get_ymax(xrange=xrange, yrange=yrange):
    """
    Return ymax highest possible probe height that will
    still hit the target
    """
    vy = 1
    ymax = max(yrange)
    while True:
        ymax0 = get_ymax_at_vy(vy)
        if ymax0 > ymax:
            ymax = ymax0
        else:
            # ymax is no longer increasing and we must
            # be done
            return ymax
        #if ymax0 < 0:
        #    return ymax
        vy += 1

#yvals = {vy: get_ymax_at_vy(vy) for vy in range(20)}
#yvals

get_ymax()

861

In [ ]:
# That's not the right answer; your answer is too low.
# Please wait one minute before trying again.
# (You guessed 861.)